In [1]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train_gene.csv', delimiter = ',')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml.csv')
# train_target = train_target[['EMB_MIC']]
# # don't touch test data, split out validation data from training data during training
# # test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_EMB/aa_data_test_pca4k.csv', delimiter = ',')
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test_gene.csv', delimiter = ',')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml.csv')
# test_target = test_target[['EMB_MIC']]

# all_data = np.concatenate((train_data, test_data), axis=0)
# all_target = pd.concat((train_target, test_target), axis=0)

# train_data, test_data, train_target, test_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

## functions

In [2]:
def data_split(aa_array, encoded_mic):
    # Encode the target variable
    # Perform stratified train-test split
    train_data, test_data, train_target, test_target = train_test_split(
        aa_array,
        encoded_mic,
        test_size=0.1,  # 10% for testing
        stratify=encoded_mic,  # Ensures the proportion of each class is preserved
        random_state=42  # For reproducibility
    )
    return train_data, test_data, train_target, test_target
def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])

def transform_array(arr):
    transformed = []
    for item in arr:
        # Only process string items that contain a comma
        if isinstance(item, str) and ',' in item:
            parts = [part.strip() for part in item.split(',')]
            transformed.extend(parts)
        else:
            transformed.append(item)
    return transformed

def data_prep_cryptic(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
    print(len(overlap))
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        else:
            aa.append([0]*len(all_snp))
        
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1
    
    mic_aa = cryptic[cryptic['ENA_RUN'].isin(overlap)]#.iloc[:,14:27]
    # mic_aa = cryptic[cryptic['ENA_RUN'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]
    # print(mic_aa.shape)
    # mic_aa['wgs_id'] = pd.Categorical(mic_aa['ENA_RUN'], categories=variants['sample_id'].unique().tolist(), ordered=True)
    # mic_aa = mic_aa.sort_values('ENA_RUN')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)
    # mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(variants['sample_id'].unique().tolist())
    mic_aa = mic_aa.sort_values(["ENA_RUN"])  ## 'sort' changed to 'sort_values'
    
    return aa_array, mic_aa

def data_prep_50k(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    # np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            # print(variants[variants['sample_id']==x]['drugs'].unique())
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
                # print('0')
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

### RIF

In [6]:
drug = 'RIF'
dr_list = ['rifampicin']

# df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')
gene_list = ['rpoA', 'rpoB', 'rpoC']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '>8':
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        # df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03125'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


### ETH


In [7]:
drug = 'ETH'
dr_list = ['ethionamide']
gene_list = ['ethA', 'inhA']
# df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')
    
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.25'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


### INH


In [8]:
drug = 'INH'
gene_list = ['inhA', 'katG','kasA']
dr_list = ['isoniazid']
# df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

# df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1.6' :
        df_emb.loc[i, f'{x}'] = '3.2'
    elif row[x] == '<=0.025':
        # df_emb.loc[i, f'{x}'] = '0.0125'
        df_emb.loc[i, f'{x}'] = '0.025'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)


# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


### RFB -- skipped


In [9]:
drug = 'RFB'
res_thresh = 0.12
dr_list = ['rifabutin']
gene_list = ['rpoA', 'rpoB', 'rpoC']
# df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>2' :
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03125'
        
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


KeyError: 'rifabutin'

### MXF


In [10]:
drug = 'MXF'
dr_list = ['moxifloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4', '2', '1', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))        
    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


(0,)
0


0it [00:00, ?it/s]


### KAN 


In [11]:
drug = 'KAN'
dr_list = ['kanamycin']
gene_list = ['rrs', 'eis']
res_thresh = 4
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>16','16.0', '8.0', '4.0', '2.0', '<=1'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>16' :
        df_emb.loc[i, f'{x}'] = '32'
    elif row[x] == '<=1':
        df_emb.loc[i, f'{x}'] = '1.0'
        # df_emb.loc[i, f'{x}'] = '0.5'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

(0,)
0
0


0it [00:00, ?it/s]


(0,)
0


0it [00:00, ?it/s]


### LEV


In [12]:
drug = 'LEV'
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['levofloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.12'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.12':
        df_emb.loc[i, f'{x}'] = '0.125
        # df_emb.loc[i, f'{x}'] = '0.06'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

SyntaxError: EOL while scanning string literal (2805265129.py, line 17)

### AMI


In [ ]:
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['amikacin']
drug = 'AMI'
# gene_list = np.unique(variants['gene'])
gene_list = ['rrs', 'eis']

res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '<=0.25'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        # df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))        
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

/tmp/ipykernel_90156/2595866171.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(306,)
306
11429


100%|██████████| 11429/11429 [00:35<00:00, 320.70it/s]


(306,)
306


100%|██████████| 124016/124016 [1:35:41<00:00, 21.60it/s]


### BDQ



In [ ]:
drug = 'BDQ'
gene_list = [ 'Rv0678', 'atpE', 'pepQ']
res_thresh = 0.25
dr_list = ['bedaquiline']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>1', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

/tmp/ipykernel_90156/1471731688.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(234,)
234
10334


100%|██████████| 10334/10334 [00:01<00:00, 8051.41it/s]


(234,)
234


100%|██████████| 11552/11552 [00:53<00:00, 214.13it/s]


### CFZ


In [ ]:
drug = 'CFZ'
dr_list = ['clofazimine']
# gene_list = np.unique(variants['gene'])
gene_list = [ 'Rv0678', 'pepQ']
res_thresh = 0.25
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

### DLM


In [ ]:
drug = 'DLM'
dr_list = ['delamanid']
res_thresh = 0.12
gene_list = ['fgd1','fbiA','fbiB','fbiC','fbiD','ddn']
# gene_list = variants['gene'].unique()
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df[f'{drug}_MIC'].isin(['>0.5', '0.5', '0.25', '0.12', '0.06', '0.03', '0.015', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>0.5' :
        df_emb.loc[i, f'{x}'] = '1'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.0078125'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'        
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'        
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
# aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

/tmp/ipykernel_90156/2468873758.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(906,)
906
8807


100%|██████████| 8807/8807 [00:06<00:00, 1306.31it/s]


(906,)
906


100%|██████████| 49653/49653 [21:10<00:00, 39.10it/s]


### EMB

In [ ]:
drug = 'EMB'
dr_list = ['ethambutol']
res_thresh = 8
gene_list = ['embB', 'embA', 'embC']
# gene_list = variants['gene'].unique()
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')

cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}-tbp.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{dr_list[0]}']))    
    
    
# with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_50k)
# with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_50k))

/tmp/ipykernel_90156/1172994841.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(1710,)
1710
11362


100%|██████████| 11362/11362 [01:07<00:00, 168.37it/s]


(1710,)
1710


100%|██████████| 122744/122744 [3:02:29<00:00, 11.21it/s] 


# sanity check with the EMB

In [33]:
remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']

In [34]:
variants = pd.read_csv('../variants_full.csv')


In [35]:
variants[~variants['sample_id'].isin(remove_from_50k)]

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
1,ERR9787047,7362,gyrA,p.Glu21Gln,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
2,ERR9787047,7585,gyrA,p.Ser95Thr,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
3,ERR9787047,8493,gyrA,p.Leu398Phe,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
4,ERR9787047,9143,gyrA,c.1842T>C,1.000000,synonymous_variant,lineage6.1.1,Sensitive,NaN
...,...,...,...,...,...,...,...,...,...
7420991,ERR718460,4328399,ethA,c.-926C>T,0.986486,upstream_gene_variant,lineage2.2.1,Other,NaN
7420992,ERR718460,4338365,whiB6,p.Cys53Arg,1.000000,missense_variant,lineage2.2.1,Other,NaN
7420993,ERR718460,4338595,whiB6,c.-75delG,1.000000,upstream_gene_variant,lineage2.2.1,Other,NaN
7420994,ERR718460,4407588,gid,c.615A>G,1.000000,synonymous_variant,lineage2.2.1,Other,NaN


#### 50k with samples removed

In [36]:

def data_prep_50k_mod(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']
    variants = variants[~variants['sample_id'].isin(remove_from_50k)]
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

In [ ]:
drug = 'EMB'
dr_list = ['ethambutol']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

gene_list = ['embB', 'embA', 'embC']
df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
# aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k_mod(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

# with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_cryptic)
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}_checks.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}_checks.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/tmp/ipykernel_90156/3746604749.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

(1710,)
1710


100%|██████████| 122619/122619 [3:01:54<00:00, 11.23it/s] 


#### removed samples

In [38]:

def data_prep_50k_mod(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    # np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    remove_from_50k = ['ERR1213824', 'ERR1213825', 'ERR1213826', 'ERR1213827', 'ERR1213828', 'ERR1213829', 'ERR1213830', 'ERR1213831', 'ERR1213832', 'ERR1213833', 'ERR1213834', 'ERR1213835', 'ERR1213836', 'ERR1213837', 'ERR1213838', 'ERR1213839', 'ERR1213840', 'ERR1213841', 'ERR1213842', 'ERR1213843', 'ERR1213844', 'ERR1213845', 'ERR1213846', 'ERR1213847', 'ERR1213848', 'ERR1213849', 'ERR1213850', 'ERR1213851', 'ERR1213852', 'ERR1213853', 'ERR1213854', 'ERR1213855', 'ERR1213856', 'ERR1213857', 'ERR1213858', 'ERR1213859', 'ERR1213860', 'ERR1213861', 'ERR1213862', 'ERR1213863', 'ERR1213864', 'ERR1213865', 'ERR1213866', 'ERR1213867', 'ERR1213868', 'ERR1213869', 'ERR1213870', 'ERR1213871', 'ERR1213872', 'ERR1213873', 'ERR1213874', 'ERR1213875', 'ERR1213876', 'ERR1213877', 'ERR1213878', 'ERR1213879', 'ERR1213880', 'ERR1213881', 'ERR1213882', 'ERR1213883', 'ERR1213884', 'ERR1213885', 'ERR1213886', 'ERR1213887', 'ERR1213888', 'ERR1213889', 'ERR1213890', 'ERR1213891', 'ERR1213892', 'ERR1213893', 'ERR1213894', 'ERR1213895', 'ERR1213896', 'ERR1213897', 'ERR1213898', 'ERR1213899', 'ERR1213900', 'ERR1213901', 'ERR1213902', 'ERR1213903', 'ERR1213904', 'ERR1213905', 'ERR1213906', 'ERR1213907', 'ERR1213908', 'ERR1213909', 'ERR1213910', 'ERR1213911', 'ERR1213912', 'ERR1213913', 'ERR1213914', 'ERR1213915', 'ERR1213916', 'ERR1213917', 'ERR1213918', 'ERR1213919', 'ERR1213920', 'ERR1213921', 'ERR1213922', 'ERR1213923', 'ERR1213924', 'ERR1213925', 'ERR1213926', 'ERR1213927', 'ERR1213928', 'ERR1213929', 'ERR1213930', 'ERR1213931', 'ERR1213932', 'ERR1213933', 'ERR1213934', 'ERR1213935', 'ERR1213936', 'ERR1213937', 'ERR1213938', 'ERR1213939', 'ERR1213940', 'ERR1213941', 'ERR1213942', 'ERR1213943', 'ERR1213944', 'ERR1213945', 'ERR1213946', 'ERR1213947', 'ERR1213948', 'ERR1213949', 'ERR1213950']
    variants = variants[variants['sample_id'].isin(remove_from_50k)]
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()
    print("len(overlap)",len(overlap))

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            
            result_ = transform_array(variants[variants['sample_id']==x]['drugs'].unique())
            # print(result_)
            # print(dr_list[0])
            if dr_list[0] in result_:
                dr.append(1)
                # print('1') 
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

In [ ]:
drug = 'EMB'
dr_list = ['ethambutol']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC3/CRyPTIC_reuse_table_20231208_cleaned+tbprofiler.csv')

gene_list = ['embB', 'embA', 'embC']
df_emb = df[df['EMB_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = 'EMB_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb['EMB_MIC'] = df_emb['EMB_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
# aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k_mod(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

# with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, aa_array_cryptic)
# with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
#     np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}_checks-removed.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}_checks-removed.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/tmp/ipykernel_52736/1811954684.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

(1710,)
1710
len(overlap) 125


100%|██████████| 125/125 [00:00<00:00, 760.72it/s]


In [ ]:
test_values = ['8', '4', '≤1', '≤1', '2', '≤1', '≤1', '2', '2', '≤1', '≤1', '4', '2', '4', '≤1', '8', '2', '2', '4', '4', '8', '≤1', '≤1', '≤1', '2', '>','8', '4', '2', '4', '2', '≤1', '2', '2', '≤1', '4', '>','8', '2', '≤1', '4', '4', '8', '4', '2', '≤1', '≤1', '≤1', '≤1', '4', '4', '≤1', '>','8', '≤1', '≤1', '2', '2', '≤1', '≤1', '2', '2', '4', '≤1', '≤1', '2', '4', '≤1', '8', '2', '2', '4', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '4', '≤1', '4', '≤1', '≤1', '≤1', '≤1', '4', '2', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '8', '2', '≤1', '8', '2', '4', '2', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '>','8', '8', '4', '4']
for i, x in enumerate(test_values):
    if x == '≤1':
        test_values[i] = '1'